<a href="https://colab.research.google.com/github/sharsulkar/H1B_LCA_outcome_prediction/blob/main/prototyping/notebooks/02_sh_build_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
observations_df=pd.read_csv('https://raw.githubusercontent.com/sharsulkar/H1B_LCA_outcome_prediction/main/reports/preprocessing_steps_observations.csv',sep='$',index_col=0,error_bad_lines=False)

In [3]:
#only pull columns that are required
required_features=list(observations_df[observations_df.preprocess_comment.isin([np.NaN,'Feature engineering','Target feature'])].index)

In [4]:
LCA_df=pd.read_excel('https://www.dol.gov/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q2.xlsx',usecols=required_features)

In [5]:
#Drop rows where CASE_STATUS not in ('Certified','Denied')
LCA_df.drop(index=LCA_df[~LCA_df.CASE_STATUS.isin(['Certified','Denied'])].index,inplace=True)
LCA_df.reset_index(inplace=True)

In [114]:
def modify_observations(df,index,columns,values,modify_action='update_values'):
  #assert - index, columns and values are string list type, 
  #columns and values are same size, for single column - value should be scalar
  #columns that have modification exist in observation_df
  #
  if modify_action=='add_row':
    df.loc[index]=values

  elif modify_action=='update_values':
    df.loc[index,columns]=values
  
  return df

In [25]:
def missing_statistics(df,column):
  return (df.shape[0]-df[column].count())*100/df.shape[0]

In [26]:
def cardinality_statistics(df,column):
  return (df.shape[0]-len(df[column].unique()))*100/df.shape[0]

### Feature engineering

In [6]:
#feature engineering steps
def date_diff(date1,date2):
  #error handling - 
    #validate input datatype is datetime
    #handle empty inputs
  return date1-date2

#### Processing_Days and Validity_days

In [7]:
#Compute feature
LCA_df['PROCESSING_DAYS']=date_diff(LCA_df.DECISION_DATE, LCA_df.RECEIVED_DATE)
LCA_df['VALIDITY_DAYS']=date_diff(LCA_df.END_DATE, LCA_df.BEGIN_DATE)

In [31]:
#update observations_df with new feature details
observations_df.loc['PROCESSING_DAYS']=[LCA_df.PROCESSING_DAYS.dtype,missing_statistics(LCA_df,'PROCESSING_DAYS'),cardinality_statistics(LCA_df,'PROCESSING_DAYS'),'New feature','Feature engineering','','','Ordinal','Ordered standardized random']
observations_df.loc['VALIDITY_DAYS']=[LCA_df.VALIDITY_DAYS.dtype,missing_statistics(LCA_df,'VALIDITY_DAYS'),cardinality_statistics(LCA_df,'VALIDITY_DAYS'),'New feature','Feature engineering','','','Ordinal','Ordered standardized random']

#### SOC_Codes

In [8]:
#Compute feature
LCA_df['SOC_CD2']=LCA_df.SOC_CODE.str.split(pat='-',n=1,expand=True)[0]
LCA_df['SOC_CD4']=LCA_df.SOC_CODE.str.split(pat='-',n=1,expand=True)[1].str.split(pat='.',n=1,expand=True)[0]
LCA_df['SOC_CD_ONET']=LCA_df.SOC_CODE.str.split(pat='-',n=1,expand=True)[1].str.split(pat='.',n=1,expand=True)[1]

In [35]:
#update observations_df with new feature details
observations_df.loc['SOC_CD2']=[LCA_df.SOC_CD2.dtype,missing_statistics(LCA_df,'SOC_CD2'),cardinality_statistics(LCA_df,'SOC_CD2'),'New feature','Feature engineering','','','Categorical','Standardized random']
observations_df.loc['SOC_CD4']=[LCA_df.SOC_CD4.dtype,missing_statistics(LCA_df,'SOC_CD4'),cardinality_statistics(LCA_df,'SOC_CD4'),'New feature','Feature engineering','','','Categorical','Standardized random']
observations_df.loc['SOC_CD_ONET']=[LCA_df.SOC_CD_ONET.dtype,missing_statistics(LCA_df,'SOC_CD_ONET'),cardinality_statistics(LCA_df,'SOC_CD_ONET'),'New feature','Feature engineering','','','Categorical','Standardized random']

#### USA_YN

In [9]:
def is_USA(country):
  if country=='UNITED STATES OF AMERICA':
    USA_YN='Y' 
  else:
    USA_YN='N'
  return USA_YN

In [10]:
#Compute feature
LCA_df['USA_YN']=LCA_df.EMPLOYER_COUNTRY.apply(is_USA)

In [39]:
#update observations_df with new feature details
observations_df.loc['USA_YN']=[LCA_df.USA_YN.dtype,missing_statistics(LCA_df,'USA_YN'),cardinality_statistics(LCA_df,'USA_YN'),'New feature','Feature engineering','','','Binary','Standardized random']

#### Employer_Worksite_YN

In [11]:
#Compute feature
LCA_df['EMPLOYER_WORKSITE_YN']='Y'
LCA_df.loc[LCA_df.EMPLOYER_POSTAL_CODE.ne(LCA_df.WORKSITE_POSTAL_CODE),'EMPLOYER_WORKSITE_YN']='N'

In [40]:
#update observations_df with new feature details
observations_df.loc['USA_YN']=[LCA_df.USA_YN.dtype,missing_statistics(LCA_df,'USA_YN'),cardinality_statistics(LCA_df,'USA_YN'),'New feature','Feature engineering','','','Binary','Standardized random']

#### OES_YN

In [12]:
#Compute feature
LCA_df['OES_YN']='Y'
LCA_df.iloc[LCA_df[~LCA_df.PW_OTHER_SOURCE.isna()].index,LCA_df.columns.get_loc('OES_YN')]='N'

In [41]:
#update observations_df with new feature details
observations_df.loc['OES_YN']=[LCA_df.OES_YN.dtype,missing_statistics(LCA_df,'OES_YN'),cardinality_statistics(LCA_df,'OES_YN'),'New feature','Feature engineering','','','Binary','Standardized random']

#### SURVEY_YEAR

In [13]:
#Compute feature
#reference https://www.interviewqs.com/ddi-code-snippets/extract-month-year-pandas
#Compute and move all 'Year' values from PW_OES_YEAR field into SURVEY_YEAR
LCA_df['SURVEY_YEAR']=pd.to_datetime(LCA_df.PW_OES_YEAR.str.split(pat='-',n=1,expand=True)[0]).dt.to_period('Y')
#Then create a series object to store only the 'YEAR' values from PW_OTHER_YEAR when OES_YN==N
PW_other_year=LCA_df[LCA_df.OES_YN=='N'].PW_OTHER_YEAR
#Rename the series and update dataframe with series object
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.update.html
PW_other_year.rename("SURVEY_YEAR",inplace=True)
LCA_df.update(PW_other_year)

In [50]:
#update observations_df with new feature details
observations_df.loc['SURVEY_YEAR']=[LCA_df.SURVEY_YEAR.dtype,missing_statistics(LCA_df,'SURVEY_YEAR'),cardinality_statistics(LCA_df,'SURVEY_YEAR'),'New feature','Feature engineering','','','Ordinal','Standardized random']

#### WAGE_ABOVE_PREVAILING_HR

In [14]:
#Compute feature
#initialize with WAGE_RATE_OF_PAY_FROM
LCA_df['WAGE_PER_HR']=LCA_df.WAGE_RATE_OF_PAY_FROM
#compute for Year
LCA_df.iloc[LCA_df[LCA_df.WAGE_UNIT_OF_PAY=='Year'].index,LCA_df.columns.get_loc('WAGE_PER_HR')]=LCA_df[LCA_df.WAGE_UNIT_OF_PAY=='Year'].WAGE_RATE_OF_PAY_FROM/2067
#compute for Month
LCA_df.iloc[LCA_df[LCA_df.WAGE_UNIT_OF_PAY=='Month'].index,LCA_df.columns.get_loc('WAGE_PER_HR')]=LCA_df[LCA_df.WAGE_UNIT_OF_PAY=='Month'].WAGE_RATE_OF_PAY_FROM/172

In [15]:
#Compute feature
#initialize with WAGE_RATE_OF_PAY_FROM
LCA_df['PW_WAGE_PER_HR']=LCA_df.PREVAILING_WAGE
#compute for Year
LCA_df.iloc[LCA_df[LCA_df.PW_UNIT_OF_PAY=='Year'].index,LCA_df.columns.get_loc('PW_WAGE_PER_HR')]=LCA_df[LCA_df.PW_UNIT_OF_PAY=='Year'].PREVAILING_WAGE/2067
#compute for Month
LCA_df.iloc[LCA_df[LCA_df.PW_UNIT_OF_PAY=='Month'].index,LCA_df.columns.get_loc('PW_WAGE_PER_HR')]=LCA_df[LCA_df.PW_UNIT_OF_PAY=='Month'].PREVAILING_WAGE/172

In [16]:
LCA_df['WAGE_ABOVE_PW_HR']=LCA_df.WAGE_PER_HR-LCA_df.PW_WAGE_PER_HR

In [51]:
#update observations_df with new feature details
observations_df.loc['WAGE_ABOVE_PW_HR']=[LCA_df.WAGE_ABOVE_PW_HR.dtype,missing_statistics(LCA_df,'WAGE_ABOVE_PW_HR'),cardinality_statistics(LCA_df,'WAGE_ABOVE_PW_HR'),'New feature','Feature engineering','','','Numerical','Standard scaling']

#### Drop columns after feature engineering

In [17]:
drop_columns=['DECISION_DATE','RECEIVED_DATE','END_DATE','BEGIN_DATE','SOC_CODE','EMPLOYER_COUNTRY','EMPLOYER_POSTAL_CODE','WORKSITE_POSTAL_CODE','PW_OTHER_SOURCE','PW_OES_YEAR','PW_OTHER_YEAR','WAGE_RATE_OF_PAY_FROM','WAGE_UNIT_OF_PAY','WAGE_PER_HR','PW_WAGE_PER_HR','PREVAILING_WAGE','PW_UNIT_OF_PAY','index']
LCA_df.drop(columns=drop_columns,inplace=True)

### Encode finalized features

#### Update preprocess action for any features that will be used as is (pre encoding)

In [ ]:
observations_df[observations_df.preprocess_action.isna()]

In [94]:
index=['VISA_CLASS', 'SOC_TITLE', 'FULL_TIME_POSITION','TOTAL_WORKER_POSITIONS', 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT','CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT','CHANGE_EMPLOYER', 'AMENDED_PETITION', 'EMPLOYER_NAME', 'NAICS_CODE','AGENT_REPRESENTING_EMPLOYER', 'WORKSITE_WORKERS', 'SECONDARY_ENTITY','PW_WAGE_LEVEL', 'TOTAL_WORKSITE_LOCATIONS', 'AGREE_TO_LC_STATEMENT','H-1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'PUBLIC_DISCLOSURE']
columns=['preprocess_action']
values='Use feature as is'
observations_df=modify_observations(observations_df,index,columns,values,modify_action='update_values')

#### Update Observations_df with Categorical class and encoding plan for any features that might have been missed earlier

In [ ]:
observations_df[(observations_df['Categorical class'].isna()) & (observations_df.preprocess_action!='Drop column')]

In [104]:
index=['TOTAL_WORKER_POSITIONS', 'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT','CHANGE_PREVIOUS_EMPLOYMENT', 'NEW_CONCURRENT_EMPLOYMENT','CHANGE_EMPLOYER', 'AMENDED_PETITION', 'WORKSITE_WORKERS','TOTAL_WORKSITE_LOCATIONS']
columns=['Categorical class','embedding']
values=['Numerical','Standard scaling']
observations_df=modify_observations(observations_df,index,columns,values,modify_action='update_values')

In [18]:
#embed categorical features


In [ ]:
#scale numerical features


In [ ]:
#separate target column and convert to binary
